In [1]:
import pandas as pd
import seaborn as sns
import time
import re
import nltk
import math
import os
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def fasta_parser(filename):
    if os.path.exists(filename):
        pass
    else:                     
        print("The file, %s, does not exist" % filename)
        
    file = open(filename,mode='r')
    rec_all = file.read()
    file.close()
    # remove all whitespace from string all_of_it
    rec_all = rec_all.replace(' ','')
    # split records by > 
    records = rec_all.split('>')
    # Parse out the headers & sequences for each record
    headers = []
    sequences = []
    for rec in records:
        s = ''
        data = rec.split('\n')
        sq = s.join(data[1:])
        if len(data[0]) > 0:
            headers.append(data[0])
   
        if (len(sq) > 0):
            sequences.append(s.join(data[1:]))
    
    return headers, sequences

In [3]:
# concatenate all relevant fasta files into one fasta file use glob
# see https://stackoverflow.com/questions/17749058/combine-multiple-text-files-into-one-text-file-using-python/17749339
# Note the code below works with no bizzare artifacts from globbing or concatenation
#read_files = glob.glob("*.fasta")

#with open("isomerase_2000.fasta", "wb") as outfile:
    #for f in read_files:
        #with open(f, "rb") as infile:
            #outfile.write(infile.read())

In [4]:
f_name = 'uniprot-abc-reviewed.fasta'
t_head, t_seqs = fasta_parser(f_name)

In [5]:
for hd in t_head[0:5]:
    print(hd)
    print()

sp|Q03518|TAP1_HUMANAntigenpeptidetransporter1OS=HomosapiensOX=9606GN=TAP1PE=1SV=2
()
sp|Q03519|TAP2_HUMANAntigenpeptidetransporter2OS=HomosapiensOX=9606GN=TAP2PE=1SV=1
()
sp|Q9LVM1|AB25B_ARATHABCtransporterBfamilymember25,mitochondrialOS=ArabidopsisthalianaOX=3702GN=ABCB25PE=1SV=1
()
sp|Q9FUT3|AB23B_ARATHABCtransporterBfamilymember23,mitochondrialOS=ArabidopsisthalianaOX=3702GN=ABCB23PE=1SV=1
()
sp|Q9NP58|ABCB6_HUMANATP-bindingcassettesub-familyBmember6,mitochondrialOS=HomosapiensOX=9606GN=ABCB6PE=1SV=1
()


In [6]:
for sq in t_seqs[0:5]:
    print(sq)
    print()

MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAESPRPGRDRDGVRVPMASSRCPAPRGCRCLPGASLAWLGTVLLLLADWVLLRTALPRIFSLLVPTALPLLRVWAVGLSRWAVLWLGACGVLRATVGSKSENAGAQGWLAALKPLAAALGLALPGLALFRELISWGAPGSADSTRLLHWGSHPTAFVVSYAAALPAAALWHKLGSLWVPGGQGGSGNPVRRLLGCLGSETRRLSLFLVLVVLSSLGEMAIPFFTGRLTDWILQDGSADTFTRNLTLMSILTIASAVLEFVGDGIYNNTMGHVHSHLQGEVFGAVLRQETEFFQQNQTGNIMSRVTEDTSTLSDSLSENLSLFLWYLVRGLCLLGIMLWGSVSLTMVTLITLPLLFLLPKKVGKWYQLLEVQVRESLAKSSQVAIEALSAMPTVRSFANEEGEAQKFREKLQEIKTLNQKEAVAYAVNSWTTSISGMLLKVGILYIGGQLVTSGAVSSGNLVTFVLYQMQFTQAVEVLLSIYPRVQKAVGSSEKIFEYLDRTPRCPPSGLLTPLHLEGLVQFQDVSFAYPNRPDVLVLQGLTFTLRPGEVTALVGPNGSGKSTVAALLQNLYQPTGGQLLLDGKPLPQYEHRYLHRQVAAVGQEPQVFGRSLQENIAYGLTQKPTMEEITAAAVKSGAHSFISGLPQGYDTEVDEAGSQLSGGQRQAVALARALIRKPCVLILDDATSALDANSQLQVEQLLYESPERYSRSVLLITQHLSLVEQADHILFLEGGAIREGGTHQQLMEKKGCYWAMVQAPADAPE
()
MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRLGGLWGLLKLRGLLGFVGTLLLPLCLATPLTVSLRALVAGASRAPPARVASAPWSWLLVGYGAAGLSWSLWAVLSPPGAQEKEQDQVNNKVLMWRLLKLSRPDLPLLVAAFFFLVLAVLGETLIPHYSGRVIDILGGDFDPHA

In [7]:
print(len(t_seqs))

4195


In [8]:
t_seqs2 = list(set(t_seqs))
print(len(t_seqs2))

3642


In [9]:
from Bio import SeqIO
records = list(SeqIO.parse(f_name, "fasta"))


In [10]:
seq_ids =[]
seqs = []
#for record in SeqIO.parse("isomerase_2000.fasta", "fasta"):
for record in SeqIO.parse(f_name, "fasta"):
    print(repr(record.seq))
    print()
    seq_ids.append(record.id)
    seqs.append(str(record.seq))

Seq('MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAESPRPGRDR...APE', SingleLetterAlphabet())
()
Seq('MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRLGGLWGLLK...AQL', SingleLetterAlphabet())
()
Seq('MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNYIRRNSIRT...KLE', SingleLetterAlphabet())
()
Seq('MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFFTDPSPSPS...LEV', SingleLetterAlphabet())
()
Seq('MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVLALPCRRRE...MER', SingleLetterAlphabet())
()
Seq('MPSLQLLQLTERGRGLVASRRKSILLAAGIVAAGGTAVYLKSRVASRRPDSSRL...TTE', SingleLetterAlphabet())
()
Seq('MNESNRELILGLSPVHLSLFNSESLLHTFNFFGVGDGQANIASNYVTASKFVIP...YID', SingleLetterAlphabet())
()
Seq('MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRVKLKSGFLP...KYS', SingleLetterAlphabet())
()
Seq('MALRGFCSADGSDPLWDWNVTWNTSNPDFTKCFQNTVLVWVPCFYLWACFPFYF...GLV', SingleLetterAlphabet())
()
Seq('MALRSFCSADGSDPLWDWNVTWHTSNPDFTKCFQNTVLTWVPCFYLWSCFPLYF...GLV', SingleLetterAlphabet())
()
Seq('MALSSFCSSDGSDPLWDWNVTWHTSNPDFTKCFQNTVLTWVPCFY

Seq('MDRRSRAMADSVLSLNKLDVGFPGMTVVRDLNLDVADGAFVSLLGPSGSGKSSV...VTG', SingleLetterAlphabet())
()
Seq('MAGLKLQAVTKSWDGKTQVIKPLTLDVADGEFIVMVGPSGCGKSTLLRMVAGLE...QRV', SingleLetterAlphabet())
()
Seq('MTINTDDQYADGESKTTISSNRRYSTSSFQDQSMEDDGINATLTNDKATLIQVW...SKE', SingleLetterAlphabet())
()
Seq('MYSGQHIKEPVTRVAGLQKSVLPMLVVDSITLFYGNRKVIDNVSFSIRPGEIIT...ENV', SingleLetterAlphabet())
()
Seq('MSHESIEKNLNFVKKLNNVNDSVLKIENLALAYDNKKVLDGINISIEKGDIVTI...IAK', SingleLetterAlphabet())
()
Seq('MDKKSSHPAGAARDILIELRNAGVYRDGRWLVRNVDLSVERGEIVTLIGPNGAG...PRA', SingleLetterAlphabet())
()
Seq('MTTDQHQEILRTEGLSKFFPGVKALDNVDFSLRRGEIMALLGENGAGKSTLIKA...IAA', SingleLetterAlphabet())
()
Seq('MPEPLVTVQSVSVTLGGNAVLSDVSLSLAPGRITTLIGPNGAGKSTLARLVLGL...NHD', SingleLetterAlphabet())
()
Seq('MLKVESISKDYREFKLREISFDVREGEHFIILGPSGSGKTVLLEIIAGIISPDR...VFP', SingleLetterAlphabet())
()
Seq('MKPKISFNNVVMRYGGFLALDRLNLDIADGEFVTVVGPSGCGKSTAMNIAAGLL...HGQ', SingleLetterAlphabet())
()
Seq('MKKPIITFNNFSFQYHSQSEPTLKGIQLTIYEGEKVLIVGPSGSG

()
Seq('MSIEIHASAKNLNLWYDSNQVLHNISLDIYKREVTAFIGPSGCGKSTFLRCFNR...DHL', SingleLetterAlphabet())
()
Seq('MQPDLNPNTTQPLIALTGIGKRFPGVQALDDCRFDLRAGEVHALMGENGAGKST...VLS', SingleLetterAlphabet())
()
Seq('MTALDTLERVETAEIEKPIMQDKQIARGVEKITKTVGVPLTENAKFGIRGVDVF...RFG', SingleLetterAlphabet())
()
Seq('MVTLDTQSIIRTVNVNFYYGRFQALTDITMDFQKNRVTALIGPSGCGKSTLLRL...RFG', SingleLetterAlphabet())
()
Seq('MNVSVNIKNVTKEYRIYRTNKERMKDALIPKHKNKTFFALDDISLKAYEGDVIG...VIK', SingleLetterAlphabet())
()
Seq('MNATTSAAAYGPLAGADLEAELAQARVADSDARDAAILEHDGSASVVPLARRRT...WAV', SingleLetterAlphabet())
()
Seq('MNTARLNQGTPLLLNAVSKHYAENIVLNQLDLHIPAGQFVAVVGRSGGGKSTLL...RHG', SingleLetterAlphabet())
()
Seq('MDNRTLFEMRSIVKAFSGVRALDGVSLAVRPGECVGLCGENGAGKSTLMKVLSG...RAA', SingleLetterAlphabet())
()
Seq('MSLLENIRMALSSVLAHKMRSILTMLGIIIGVGSVIVVVAVGQGGEQMLKQSIS...RYE', SingleLetterAlphabet())
()
Seq('MPYLLEMKNITKTFGSVKAIDNVSLRLNAGEIVSLCGENGSGKSTLMKVLCGIY...QSV', SingleLetterAlphabet())
()
Seq('MQITVSNLKKSYGGSTVLDVESLTFESGKITGIIGPNGAGKT

Seq('MQKPIIEFRNVSKKFGNKLPINNVSFTVKKNNITTLIGPNGAGKTTIVRLMLGL...HKH', SingleLetterAlphabet())
()
Seq('MENIVNLKNIFVFYGKKKILNNISFKISHGKIITILGPNGSGKSTLARVILGLI...HNH', SingleLetterAlphabet())
()
Seq('MSFLHVDHVTHTYFSIKEKTTAVRDIHFDAEKGDFISFLGPSGCGKTTLLSIIA...RNH', SingleLetterAlphabet())
()
Seq('MDSFSTSIWVNDVTVRYNNGHTAIHNMTFSLNSGTICALVGVNGSGKSTLFKSI...KRD', SingleLetterAlphabet())
()
Seq('MKKLIELKGVSRTYGNGDQTRTVLKNVDLTIVAGEMVAIIGASGSGKSTLMNIM...ASE', SingleLetterAlphabet())
()
Seq('MPQQIELRNITLQAAQPLVHGVSLTLQRGRVLALVGGSGSGKSLTCAATLGILP...LAS', SingleLetterAlphabet())
()
Seq('MKTIISAKDVHLSYGDYEALHGINLNFKEKELTALIGPSGCGKSTFLRCLNRMN...RFG', SingleLetterAlphabet())
()
Seq('MQNVLTQRLTNDEQQKTLKTLIEKNLEIKTMNKFDIENLDLYYGENQALKAINL...DFG', SingleLetterAlphabet())
()
Seq('MIVTETIIRFENVTKQFDNDPPVLDNVSFEIEKGKFYTLLGPSGCGKTTILRLI...EVQ', SingleLetterAlphabet())
()
Seq('MPDKPDRNAIEVVNVSKIFGSGEGQVAALDKVSVSIRENEFFTLLGPSGCGKTT...LRD', SingleLetterAlphabet())
()
Seq('MKNIEKSEIIISLVDVDKEFGDKKVLDQINLDIKRGDFVTLLGPS

Seq('MSQQENGDVATELIENRLSFSRIPRISLHVRDLSIVASKTNTTLVNTFSMDLPS...IKW', SingleLetterAlphabet())
()
Seq('MQPIISFEQFNFQYKHAAQPTVKDVTFHIYPGEKVLIAGRSGSGKSTLAHCMNG...EGV', SingleLetterAlphabet())
()
Seq('MLQVKNLSFKYPKHQNNVLNQISFNVQDGCHLAVIGHNGSGKSTLVKLLGGFLK...HLK', SingleLetterAlphabet())
()
Seq('MSSKSASKLKREAKKAERLAAKGESVKPSKKNGTKNGKDKEVDGVTKDLSELST...HLI', SingleLetterAlphabet())
()
Seq('MPYLLEMKDITKQFGVVKAVDNISLTLEAGQVLSLCGENGSGKSTLMKVLCGIY...VTS', SingleLetterAlphabet())
()
Seq('MEGSWSYKLLYVFLCIVLGILYGIANPILLAQGLGFIFPITSSNGRAVDSIYSL...SVR', SingleLetterAlphabet())
()
Seq('MRYFALTDLTKPTAKFTTDAMELRQISNAYLNQAQAYLQKGLKQLKKDYKNAIL...NWI', SingleLetterAlphabet())
()
Seq('MDMVAHAEINNLSFVYADENEKALQHISLSVQKGEFIALAGGSGSGKTTLLKHL...LHS', SingleLetterAlphabet())
()
Seq('MPIILENVSFFYSKNTPLETAALKDVNLRIEKGEFVGILGEKGAGKSTLIKLFN...ETD', SingleLetterAlphabet())
()
Seq('MLFAALRDVQWRKRRLVIAIVSTGLVFAMTLVLTGLVNGFRVEAERTVDSMGVD...GGP', SingleLetterAlphabet())
()
Seq('MPLISIQNAFLAFSDLEILKNAVLYINKKERISLIGKNGAGKSTL

()
Seq('MTDAALALQAENLRMTFGETKALDDVSLSVKPGEMVALIGPSGSGKSTLLRVAA...GDA', SingleLetterAlphabet())
()
Seq('MLRIDKLTKRFGDNIAVNAATLDVDKPCMIGIIGRSGAGKSTLLRMINRLSDAT...ATA', SingleLetterAlphabet())
()
Seq('MIEFRNVSKVYPNGTKGLNNINLKIQKGEFVVMVGLSGAGKSTLLRSVNRLHEI...AVK', SingleLetterAlphabet())
()
Seq('MIVFNNVNKVWPNGKQVLKNINLEINKGELVAVIGLSGAGKTTLLKTINKINDI...YGE', SingleLetterAlphabet())
()
Seq('MGQALRKLTAADYPVVVQEPRKKVLAVKGLVKAYKSQHRVLDNINFELHAGEFV...LLH', SingleLetterAlphabet())
()
Seq('MPQRPEAARAGPVAGPDAASKPAPGPALTLRGAGRAYGPVRALRPLTLEIRRGE...VPP', SingleLetterAlphabet())
()
Seq('MTPILAAEALTYAFPGGVKALDDLSLAVPQGESLAILGPNGAGKSTLLLHLNGT...TRR', SingleLetterAlphabet())
()
Seq('MPMDEGDAQGSLLLEWKQLNYYVPAQEQSNYSFWNECRKQRELGILHDVSGHLK...GYY', SingleLetterAlphabet())
()
Seq('MSIVMQLQDVAESTRLGPLSGEVRAGEILHLVGPNGAGKSTLLARMAGMTSGKG...LTI', SingleLetterAlphabet())
()
Seq('MITTSGLTKVYRSRGREVTALDGVDLHVREGEVYGVIGQSGAGKSSLIRCVNLL...GAK', SingleLetterAlphabet())
()
Seq('MISFNNVSKVYETGGQSVHAVEDVTLSVEKGEIFGIIGFSGA

()
Seq('MIEIRNLSQRFEGPRGWIEALHNVNLTIPQGEVFGIIGRSGAGKSTLVRTINLL...YVE', SingleLetterAlphabet())
()
Seq('MTQLFDTLGFIETSAVRAGAAARDAHGAHQEAATRADVAAVSLVHVGKVFATPR...GYV', SingleLetterAlphabet())
()
Seq('MTQLFDTLGFIDTSAARTGAAAATHDAAATSTEPAGPAGAAAVSLEQVGKVFAT...GYV', SingleLetterAlphabet())
()
Seq('MKKEEWILPYIKQNARLFVLVIFLGAVTIFSAAFLMFTSGFLISKAATRPENIL...PVK', SingleLetterAlphabet())
()
Seq('MNKLRQKYLQKWLRAQQEPIKKLMRANIVLATLSSFILVAQTYFLATLLDKLIM...DIQ', SingleLetterAlphabet())
()
Seq('MAIHNLACVRDERQLFSGLNEEFAPGDGVQIEGSNGAGKTLLLRILAGLTPPDA...CFG', SingleLetterAlphabet())
()
Seq('MITFQNVNKHYGDFHVLKQINLQIEKGEVVVIIGPSGSGKSTLLRCINRLESIN...LNH', SingleLetterAlphabet())
()
Seq('MPKDKAVGIQVSQVSKQFGSFQAVKDVDLTVETGSLVALLGPSGSGKSTLLRLI...YSI', SingleLetterAlphabet())
()
Seq('MITVTNARKNYGNFAALDDVTIEIPSGELTALLGPSGSGKSTLLRSIAGLEALD...PTQ', SingleLetterAlphabet())
()
Seq('MSIVMQLQDVAESTRLGPLSGEVRAGEILHLVGPNGAGKSTLLARMAGMTSGKG...STI', SingleLetterAlphabet())
()
Seq('MSQLMQLKDVAESTRLGPLSGEVSAGEILHLVGPNGAGKSTL

Seq('MEITFQKVEHRYQYKTPFERRALYDVDVSFPSGGYYAIIGHTGSGKSTMIQHLN...CSS', SingleLetterAlphabet())
()
Seq('MKITLNSVSFRYNGDYVLKDVNAEFETGKIYVVVGKNGSGKTTLLKILAGLLEA...DTV', SingleLetterAlphabet())
()
Seq('MEITFQKVEHRYQYKTPFERRALYDVDVSFPSGGYYAIIGHTGSGKSTMIQHLN...CSS', SingleLetterAlphabet())
()
Seq('MEITFQKVEHRYQYKTPFERRALYDVDVSFPSGGYYAIIGHTGSGKSTMIQHLN...CSS', SingleLetterAlphabet())
()
Seq('MEITFQKVEHRYQHKTPFERRALYDVDVSFPSGGYYAIIGHTGSGKSTMIQHLN...CSS', SingleLetterAlphabet())
()
Seq('MPIKIENLTYTYMPGTPFEKKALDNVNITIEDGEFAVFIGHTGSGKSTLIQQIN...AKR', SingleLetterAlphabet())
()
Seq('MSIKFKKVDYIYSPETPMEKKGLDNVSFELADNSFVALIGHTGSGKSTLMQHFN...FYE', SingleLetterAlphabet())
()
Seq('MAIAFEHVTYTYQAGTPMAHTALTDVSLTVPDRGYLAIIGHTGSGKSTLIQQLN...DHV', SingleLetterAlphabet())
()
Seq('MEDYILKVEELNYNYSDGTHALKGINMNIKRGEVTAILGGNGVGKSTLFQNFNG...FND', SingleLetterAlphabet())
()
Seq('MAIKFENVSYVYSPGSPLEAIGLDQLNFSLEEGKFIALVGHTGSGKSTLMQHFN...EHE', SingleLetterAlphabet())
()
Seq('MSIEFKNVDYIYAPGTPFQTQGLTNISFKIRSGSFVAIAGHTGSG

In [11]:
print(len(seq_ids))
print()
print(len(seqs))

4195
()
4195


In [12]:
seqs[0:3]

['MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAESPRPGRDRDGVRVPMASSRCPAPRGCRCLPGASLAWLGTVLLLLADWVLLRTALPRIFSLLVPTALPLLRVWAVGLSRWAVLWLGACGVLRATVGSKSENAGAQGWLAALKPLAAALGLALPGLALFRELISWGAPGSADSTRLLHWGSHPTAFVVSYAAALPAAALWHKLGSLWVPGGQGGSGNPVRRLLGCLGSETRRLSLFLVLVVLSSLGEMAIPFFTGRLTDWILQDGSADTFTRNLTLMSILTIASAVLEFVGDGIYNNTMGHVHSHLQGEVFGAVLRQETEFFQQNQTGNIMSRVTEDTSTLSDSLSENLSLFLWYLVRGLCLLGIMLWGSVSLTMVTLITLPLLFLLPKKVGKWYQLLEVQVRESLAKSSQVAIEALSAMPTVRSFANEEGEAQKFREKLQEIKTLNQKEAVAYAVNSWTTSISGMLLKVGILYIGGQLVTSGAVSSGNLVTFVLYQMQFTQAVEVLLSIYPRVQKAVGSSEKIFEYLDRTPRCPPSGLLTPLHLEGLVQFQDVSFAYPNRPDVLVLQGLTFTLRPGEVTALVGPNGSGKSTVAALLQNLYQPTGGQLLLDGKPLPQYEHRYLHRQVAAVGQEPQVFGRSLQENIAYGLTQKPTMEEITAAAVKSGAHSFISGLPQGYDTEVDEAGSQLSGGQRQAVALARALIRKPCVLILDDATSALDANSQLQVEQLLYESPERYSRSVLLITQHLSLVEQADHILFLEGGAIREGGTHQQLMEKKGCYWAMVQAPADAPE',
 'MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRLGGLWGLLKLRGLLGFVGTLLLPLCLATPLTVSLRALVAGASRAPPARVASAPWSWLLVGYGAAGLSWSLWAVLSPPGAQEKEQDQVNNKVLMWRLLKLSRPDLPLLVAAFFFLVLAVLGETLIPHYSGRVIDILGGDFD

In [13]:
seqID = []
for item in records:
    seqID.append(item.id)
    print(item.id)
    print()

sp|Q03518|TAP1_HUMAN
()
sp|Q03519|TAP2_HUMAN
()
sp|Q9LVM1|AB25B_ARATH
()
sp|Q9FUT3|AB23B_ARATH
()
sp|Q9NP58|ABCB6_HUMAN
()
sp|Q94FB9|AB1D_ARATH
()
sp|Q5A762|MLT1_CANAL
()
sp|Q9UNQ0|ABCG2_HUMAN
()
sp|P33527|MRP1_HUMAN
()
sp|O35379|MRP1_MOUSE
()
sp|Q8CG09|MRP1_RAT
()
sp|P78363|ABCA4_HUMAN
()
sp|Q8HXQ5|MRP1_BOVIN
()
sp|Q9NP78|ABCB9_HUMAN
()
sp|Q9C8G9|AB1C_ARATH
()
sp|Q7GB25|AB5C_ARATH
()
sp|O15438|MRP3_HUMAN
()
sp|Q6UR05|MRP1_CANLF
()
sp|Q42093|AB2C_ARATH
()
sp|Q9LK64|AB3C_ARATH
()
sp|O15439|MRP4_HUMAN
()
sp|Q9ZR72|AB1B_ARATH
()
sp|P39109|YCFI_YEAST
()
sp|Q7DM58|AB4C_ARATH
()
sp|Q864R9|MRP1_MACFA
()
sp|O80725|AB4B_ARATH
()
sp|Q0TJD9|MSBA_ECOL5
()
sp|Q2SJY7|POTA_HAHCH
()
sp|Q97KS6|POTA_CLOAB
()
sp|Q8YM92|POTA_NOSS1
()
sp|Q4L5B3|POTA_STAHJ
()
sp|Q6HHI7|SSUB_BACHK
()
sp|Q9HYF9|SSUB2_PSEAE
()
sp|Q9ZNB0|Y742_STRCO
()
sp|O34979|YVRO_BACSU
()
sp|Q8UBB7|UGPC2_AGRFC
()
sp|Q50801|Y583_METTM
()
sp|Q17320|WHITE_CERCA
()
sp|Q2W4W1|ZNUC_MAGSA
()
sp|Q2YJH4|ZNUC_BRUA2
()
sp|Q4QND5|ZNUC_HAEI8
()
sp|A0KPH6

sp|Q9SIT6|AB5G_ARATH
()
sp|Q09429|ABCC8_RAT
()
sp|Q55BC0|ABCA8_DICDI
()
sp|Q8SRV5|ABCF3_ENCCU
()
sp|Q8NE71|ABCF1_HUMAN
()
sp|Q55DW4|ABCG1_DICDI
()
sp|Q2GFZ6|ZNUC_EHRCR
()
sp|A1U776|ZNUC_MARHV
()
sp|O84421|Y416_CHLTR
()
sp|P63396|Y1312_MYCBO
()
sp|P9WQI4|Y2564_MYCTO
()
sp|Q5X498|MSBA_LEGPA
()
sp|Q6LH11|RBSA_PHOPR
()
sp|Q89WG0|UGPC_BRADU
()
sp|Q55740|Y385_SYNY3
()
sp|P44661|Y360_HAEIN
()
sp|Q8U648|SSUB2_AGRFC
()
sp|P9WQK1|Y986_MYCTU
()
sp|P71082|YGAD_BACSU
()
sp|Q7NRX5|UGPC_CHRVO
()
sp|O32151|YURJ_BACSU
()
sp|P38735|VMR1_YEAST
()
sp|P0A9U0|YBBA_SHIFL
()
sp|Q07UI9|UGPC_RHOP5
()
sp|Q49430|Y390_MYCGE
()
sp|P0CZ42|Y208_STRP3
()
sp|Q4UMZ3|Y214_RICFE
()
sp|P33982|Y3926_AZOC5
()
sp|Q5HCL3|Y2708_STAAC
()
sp|P9WQI6|Y2326_MYCTO
()
sp|Q7NNW9|Y289_GLOVI
()
sp|Q97X60|Y1893_SACS2
()
sp|P47433|Y187_MYCGE
()
sp|Q57213|Y1474_HAEIN
()
sp|Q57RH4|MODC_SALCH
()
sp|Q63VX7|MSBA_BURPS
()
sp|Q9JXR3|MSBA_NEIMB
()
sp|P75059|POTA_MYCPN
()
sp|Q2YAD6|POTA_NITMU
()
sp|Q1JGY7|POTA_STRPD
()
sp|Q8DZJ0|POTA_STRA5
()
sp|Q6

()
sp|Q87ZE0|RGMG_PSESM
()
sp|P63366|PSTB_SALTI
()
sp|Q1GUY1|PSTB_SPHAL
()
sp|Q0TAY4|PSTB_ECOL5
()
sp|Q8DGZ3|PSTB_THEEB
()
sp|Q31BF6|PSTB_PROM9
()
sp|Q7M9G3|PSTB_WOLSU
()
sp|Q8CPA1|PSTB_STAES
()
sp|Q7MEV1|RBSA_VIBVY
()
sp|Q1M5X4|RBSA2_RHIL3
()
sp|Q1JGL2|PSTB2_STRPD
()
sp|Q8DU23|PSTB2_STRMU
()
sp|Q2RWU0|PSTB_RHORT
()
sp|Q1BWN5|RBSA1_BURCA
()
sp|P32568|SNQ2_YEAST
()
sp|P96119|TROD_TREPA
()
sp|Q8CMU4|PHNC_STAES
()
sp|Q046T0|PHNC_LACGA
()
sp|Q07757|NODJ_AZOC5
()
sp|Q65HC0|PSTB1_BACLD
()
sp|Q834B4|PSTB1_ENTFA
()
sp|Q5V225|PSTB1_HALMA
()
sp|P46903|NATA_BACSU
()
sp|P0A2V0|NDVA_AGRFC
()
sp|Q1JBJ5|PSTB1_STRPB
()
sp|Q6FCW7|PSTB_ACIAD
()
sp|Q480N3|MSBA2_COLP3
()
sp|Q8EAN3|MODC_SHEON
()
sp|P0AAH5|SAPD_ECOL6
()
sp|Q4QJQ0|MODC_HAEI8
()
sp|Q8ZQR6|MODC_SALTY
()
sp|Q1BUV6|MSBA_BURCA
()
sp|Q3JUI6|MSBA_BURP1
()
sp|Q03JH1|POTA_STRTD
()
sp|Q0T5R2|POTA_SHIF8
()
sp|P69877|POTA_SHIFL
()
sp|Q92DL6|POTA_LISIN
()
sp|Q7MKU3|POTA_VIBVY
()
sp|P31060|MODF_ECOLI
()
sp|Q96J65|MRP9_HUMAN
()
sp|Q98QE1|POTA_MYCPU
()
sp|Q

sp|Q7ANN4|PRSD_RHIME
()
sp|Q9K8N1|PHNC3_BACHD
()
sp|Q9I2N4|MODC_PSEAE
()
sp|Q2W1R8|MODC_MAGSA
()
sp|Q21BF6|MODC_RHOPB
()
sp|Q6QJE2|SULP2_CHLRE
()
sp|P45288|SAPD_HAEIN
()
sp|Q2FKB7|PHNC_STAA3
()
sp|Q62H59|PHNC_BURMA
()
sp|Q57SD6|PHNT_SALCH
()
sp|Q5P4W2|MODC_AROAE
()
sp|Q5LT05|POTA_RUEPO
()
sp|Q58418|PSTB_METJA
()
sp|P63362|PSTB_BRUSU
()
sp|Q399X3|RGMG1_BURL3
()
sp|Q4FQD1|PSTB_PSYA2
()
sp|Q7VZ66|PSTB_BORPE
()
sp|Q4ZRC6|RGMG_PSEU2
()
sp|Q10YP7|PSTB_TRIEI
()
sp|Q0HH38|PSTB_SHESM
()
sp|Q609Z8|PSTB_METCA
()
sp|A0K6Q0|RGMG1_BURCH
()
sp|Q13FD9|RGMG_PARXL
()
sp|Q9LC44|TAGH_STAAU
()
sp|Q0SK28|SSUB1_RHOJR
()
sp|Q0RKH4|SSUB2_FRAAA
()
sp|Q8NR42|SSUB_CORGL
()
sp|Q0D9V6|STAR1_ORYSJ
()
sp|Q21JK3|CCMA_SACD2
()
sp|Q3JNJ9|ARAG_BURP1
()
sp|Q0I2Z4|FBPC_HAES1
()
sp|Q8KFE9|MACB_CHLTE
()
sp|P33593|NIKD_ECOLI
()
sp|P36497|PEDD_PEDAC
()
sp|Q21GS5|MODC_SACD2
()
sp|Q2SVU4|RBSA1_BURTA
()
sp|Q9L0Q1|MSIK_STRCO
()
sp|Q2YIN5|MODC_BRUA2
()
sp|Q48479|RFBB2_KLEPN
()
sp|O83321|RFUB_TREPA
()
sp|Q57AC2|PSTB_BRUAB
()
sp|Q9XB

sp|Q57180|Y1051_HAEIN
()
sp|O29527|Y731_ARCFU
()
sp|P57445|UUP_BUCAI
()
sp|Q8XDM1|XYLG_ECO57
()
sp|P28009|UVRA_PROMI
()
sp|Q8FW07|UGPC_BRUSU
()
sp|A5VU87|Y2548_BRUO2
()
sp|Q8TQ05|Y1747_METAC
()
sp|Q8TQW9|Y1418_METAC
()
sp|Q93D97|Y1934_STRMU
()
sp|Q9RRL9|Y2469_DEIRA
()
sp|O57872|Y132_PYRHO
()
sp|Q99QV7|Y2684_STAAM
()
sp|Q6G868|Y1788_STAAS
()
sp|Q7A0J1|Y1806_STAAW
()
sp|Q8PSR0|Y3016_METMA
()
sp|Q88ZZ2|Y149_LACPL
()
sp|Q48TC3|PSTB1_STRPM
()
sp|P0CZ37|PSTB1_STRPQ
()
sp|Q6KIP2|POTA_MYCMO
()
sp|Q31ZK0|POTA_SHIBS
()
sp|Q32EY4|POTA_SHIDS
()
sp|O83658|POTA_TREPA
()
sp|P68579|SUNT_BACSU
()
sp|Q49VI3|TAGH_STAS1
()
sp|Q88ZH4|TAGH_LACPL
()
sp|Q1M7A6|SSUB2_RHIL3
()
sp|P0AAI1|SSUB_ECOLI
()
sp|Q7CFR2|XYLG_YERPE
()
sp|O34863|UVRA_BACSU
()
sp|Q88YI7|UVRA_LACPL
()
sp|Q8G0I9|UVRA_BRUSU
()
sp|P9WQK7|UVRA_MYCTU
()
sp|Q9Z985|UVRA_CHLPN
()
sp|Q1XDG5|YCF38_PYRYE
()
sp|Q98PL2|UVRA_MYCPU
()
sp|Q8PY27|Y1037_METMA
()
sp|P77504|YBBP_ECOLI
()
sp|Q9ZB70|Y468A_MYCGE
()
sp|Q6CZ34|UGPC_PECAS
()
sp|Q8U3E0|Y528_PYRFU
()
s

sp|Q57GZ7|MALK_SALCH
()
sp|P77257|LSRA_ECOLI
()
sp|Q8YUV1|PHNC1_NOSS1
()
sp|Q02QM1|PHNC1_PSEAB
()
sp|Q20Y31|PHNC2_RHOPB
()
sp|Q7A1Z1|PHNC_STAAW
()
sp|Q1BWI2|NODI_BURCA
()
sp|Q8ZPK4|OSMV_SALTY
()
sp|Q74KF9|PSTB1_LACJO
()
sp|P9WQL0|PSTB1_MYCTO
()
sp|Q87R16|MSBA_VIBPA
()
sp|Q4ZRT7|PSTB1_PSEU2
()
sp|Q8D928|BTUD_VIBVU
()
sp|Q2QLC5|CFTR_CARPS
()
sp|Q07DX5|CFTR_NOMLE
()
sp|Q9Z651|CCMA_TATCI
()
sp|Q0A808|CCMA_ALKEH
()
sp|Q9KSD1|MGLA_VIBCH
()
sp|Q0T2X5|MGLA_SHIF8
()
sp|Q0WJP9|LSRA_YERPE
()
sp|Q3YUN6|PHNC_SHISS
()
sp|Q47F10|PHNC_DECAR
()
sp|Q38YC2|PSTB1_LACSS
()
sp|Q3SFZ6|MSBA_THIDA
()
sp|P63359|MSBA_SALTY
()
sp|Q12M46|MSBA_SHEDO
()
sp|Q8XXB6|MSBA_RALSO
()
sp|Q3Z3K7|MSBA_SHISS
()
sp|Q87EF0|MSBA_XYLFT
()
sp|Q9FVV9|NAP5_ARATH
()
sp|Q00752|MSMK_STRMU
()
sp|O74676|CDR4_CANAX
()
sp|Q73HX8|CCMA_WOLPM
()
sp|Q7VN12|CCMA_HAEDU
()
sp|O14134|ELF1_SCHPO
()
sp|Q6GJL2|METN1_STAAR
()
sp|Q8ZRM9|METN1_SALTY
()
sp|Q88RL5|METN1_PSEPK
()
sp|Q8Y4L8|METN2_LISMO
()
sp|Q7A6M2|METN2_STAAN
()
sp|Q3KJS6|METN2_PSEPF
()
sp|

sp|P21447|MDR1A_MOUSE
()
sp|Q47C66|LOLD_DECAR
()
sp|Q312H8|LOLD_DESAG
()
sp|Q1RD37|LOLD_ECOUT
()
sp|P13568|MDR_PLAFF
()
sp|Q2GHT4|LOLD_EHRCR
()
sp|O53645|MDREP_MYCTU
()
sp|Q5FFC0|LOLD_EHRRG
()
sp|H6TB12|MDR_STABO
()
sp|P21441|MDR_LEITA
()
sp|Q2A4V5|LOLD_FRATH
()
sp|Q39T41|LOLD_GEOMG
()
sp|P45247|LOLD_HAEIN
()
sp|Q0I3C2|LOLD_HAES1
()
sp|Q8F6L8|LOLD_LEPIN
()
sp|Q65TB7|LOLD_MANSM
()
sp|Q0APW8|LOLD_MARMM
()
sp|Q5F8K2|LOLD_NEIG1
()
sp|Q2GE91|LOLD_NEOSM
()
sp|Q1QLB0|LOLD_NITHX
()
sp|Q3SRG8|LOLD_NITWN
()
sp|Q2G7G7|LOLD_NOVAD
()
sp|Q9CN78|LOLD_PASMU
()
sp|Q3A558|LOLD_PELCD
()
sp|Q7MU65|LOLD_PORGI
()
sp|Q48KI4|LOLD_PSE14
()
sp|Q4KFA2|LOLD_PSEF5
()
sp|Q3IL62|LOLD_PSEHT
()
sp|Q3K9F9|LOLD_PSEPF
()
sp|Q88KY4|LOLD_PSEPK
()
sp|Q4ZV73|LOLD_PSEU2
()
sp|Q2K9R2|LOLD_RHIEC
()
sp|Q2IWV8|LOLD_RHOP2
()
sp|Q3IZT1|LOLD_RHOS4
()
sp|Q4UMZ7|LOLD_RICFE
()
sp|Q68W38|LOLD_RICTY
()
sp|Q168E3|LOLD_ROSDO
()
sp|Q1GFI8|LOLD_RUEST
()
sp|Q57QD7|LOLD_SALCH
()
sp|Q5PGR6|LOLD_SALPA
()
sp|Q080S4|LOLD_SHEFN
()
sp|Q0HVQ0|LOLD_SH

sp|Q8RHL0|ECFA1_FUSNN
()
sp|Q74L62|ECFA1_LACJO
()
sp|Q032H4|ECFA1_LACLS
()
sp|Q9CIS9|ECFA1_LACLA
()
sp|Q035B2|ECFA1_LACP3
()
sp|Q88XV2|ECFA1_LACPL
()
sp|Q1WSB8|ECFA1_LACS1
()
sp|Q03ZL6|ECFA1_LEUMM
()
sp|Q4A5A5|ECFA1_MYCS5
()
sp|Q4A8A2|ECFA1_MYCH7
()
sp|Q8ETV7|ECFA1_OCEIH
()
sp|Q4AA75|ECFA1_MYCHJ
()
sp|Q50294|ECFA1_MYCPN
()
sp|Q98QH5|ECFA1_MYCPU
()
sp|Q6GEL3|ECFA1_STAAR
()
sp|Q7A470|ECFA1_STAAN
()
sp|Q8DWR3|ECFA1_STRA5
()
sp|Q8DRR9|ECFA1_STRMU
()
sp|Q49ZE0|ECFA1_STAS1
()
sp|Q48QM2|ECFA1_STRPM
()
sp|Q1JJC9|ECFA1_STRPC
()
sp|P0CZ29|ECFA1_STRPQ
()
sp|Q8DMX9|ECFA1_STRR6
()
sp|Q03I82|ECFA1_STRTD
()
sp|Q5LXJ3|ECFA1_STRT1
()
sp|Q927N9|ECFA2_LISIN
()
sp|Q71WH8|ECFA2_LISMF
()
sp|A0ALT6|ECFA2_LISW6
()
sp|Q035B3|ECFA2_LACP3
()
sp|Q6F1W4|ECFA2_MESFL
()
sp|Q2FRT7|ECFA2_METHJ
()
sp|Q1WSB9|ECFA2_LACS1
()
sp|P47426|ECFA2_MYCGE
()
sp|Q6MSQ2|ECFA2_MYCMS
()
sp|Q98QH4|ECFA2_MYCPU
()
sp|Q2FER8|ECFA2_STAA3
()
sp|Q5HM28|ECFA2_STAEQ
()
sp|Q8CRI7|ECFA2_STAES
()
sp|Q4L884|ECFA2_STAHJ
()
sp|Q2YYM5|ECFA2_STAAB
()


sp|Q8PNN4|CYSA_XANAC
()
sp|Q7VI92|METN_HELHP
()
sp|Q5HV18|METN_CAMJR
()
sp|Q2FII2|METN2_STAA3
()
sp|Q6G475|HMUV_BARHE
()
sp|Q1J255|HMUV_DEIGD
()
sp|P12622|CHVD_RHIRD
()
sp|Q4HVU7|ATM1_GIBZE
()
sp|Q00830|ABCX_TRICV
()
sp|O70127|ABCBB_RAT
()
sp|Q3JHM1|HMUV_BURP1
()
sp|Q88DY1|HMUV_PSEPK
()
sp|P0C886|LSRA_SALCH
()
sp|Q8EB59|HMUV_SHEON
()
sp|Q5YVL8|HMUV_NOCFA
()
sp|Q1GJU0|HMUV_RUEST
()
sp|Q2SE49|CCMA_HAHCH
()
sp|Q87Z03|CCMA_PSESM
()
sp|P0AAF7|ARTP_ECOL6
()
sp|Q4WWW3|ATRI_ASPFU
()
sp|Q5PI55|CCMA1_SALPA
()
sp|Q2SZC2|ARAG1_BURTA
()
sp|Q52402|AARD_PROST
()
sp|Q8GU88|AB39G_ORYSJ
()
sp|Q8GU84|AB48G_ORYSJ
()
sp|Q8VZZ4|AB6C_ARATH
()
sp|Q8WWZ4|ABCAA_HUMAN
()
sp|E9PX95|ABCAH_MOUSE
()
sp|Q9NUQ8|ABCF3_HUMAN
()
sp|Q5R9Z5|ABCF3_PONAB
()
sp|Q578K3|FBPC_BRUAB
()
sp|P0A9W3|ETTA_ECOLI
()
sp|Q0VR80|CCMA_ALCBS
()
sp|Q60AB3|CCMA_METCA
()
sp|Q0BV49|CCMA_GRABC
()
sp|O94489|EF3_SCHPO
()
sp|Q1QVQ7|METN_CHRSD
()
sp|Q6D1C4|METN3_PECAS
()
sp|Q2RWA3|METN_RHORT
()
sp|Q2YWP2|METN2_STAAB
()
sp|Q9Z8Q8|METN_CHLPN
()
sp|P575

sp|Q7XA72|AB21G_ARATH
()
sp|Q8LGU1|AB8C_ARATH
()
sp|Q8N139|ABCA6_HUMAN
()
sp|Q91V24|ABCA7_MOUSE
()
sp|Q54VC1|ABCCF_DICDI
()
sp|O60706|ABCC9_HUMAN
()
sp|Q1CJS9|FBPC_YERPN
()
sp|Q9TKX3|CYSA_NEPOL
()
sp|Q83KP2|ARAG_SHIFL
()
sp|Q92WJ0|FBPC1_RHIME
()
sp|P0A9R7|FTSE_ECOLI
()
sp|A5U7B7|FTSE_MYCTA
()
sp|O05779|FTSE_MYCTU
()
sp|O34814|FTSE_BACSU
()
sp|P0A9S0|FTSE_SHIFL
()
sp|Q73F67|ECFA1_BACC1
()
sp|Q6HPN0|ECFA1_BACHK
()
sp|Q65P77|ECFA1_BACLD
()
sp|Q8XHV2|ECFA1_CLOPE
()
sp|P40735|ECFA1_BACSU
()
sp|Q890R2|ECFA1_CLOTE
()
sp|Q97EK8|ECFA1_CLOAB
()
sp|A0PXX7|ECFA1_CLONN
()
sp|Q5L3R0|ECFA1_GEOKA
()
sp|Q5FM63|ECFA1_LACAC
()
sp|A3DJK3|ECFA1_HUNT2
()
sp|Q03PY5|ECFA1_LACBA
()
sp|Q1GBJ0|ECFA1_LACDA
()
sp|Q04BY7|ECFA1_LACDB
()
sp|Q045Z8|ECFA1_LACGA
()
sp|A2RI01|ECFA1_LACLM
()
sp|Q927N8|ECFA1_LISIN
()
sp|Q38UT9|ECFA1_LACSS
()
sp|Q71WH7|ECFA1_LISMF
()
sp|Q8Y454|ECFA1_LISMO
()
sp|A0ALT7|ECFA1_LISW6
()
sp|Q6F1W5|ECFA1_MESFL
()
sp|Q2FNX9|ECFA1_METHJ
()
sp|Q7NAQ6|ECFA1_MYCGA
()
sp|Q2SRI1|ECFA1_MYCCT
()
sp|P47425

## Create pandas dataframe using seqID & sequences

In [14]:
# create dataframe of 2000 additional isomerase sequences
#d_iso = {'name':seq_ids,'sequence':seqs}
d_1 = {'name':seqID,'sequence':t_seqs}
df = pd.DataFrame(d_1)
df.head()

,name,sequence
0,sp|Q03518|TAP1_HUMAN,MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAE...
1,sp|Q03519|TAP2_HUMAN,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...
2,sp|Q9LVM1|AB25B_ARATH,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...
3,sp|Q9FUT3|AB23B_ARATH,MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFF...
4,sp|Q9NP58|ABCB6_HUMAN,MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVL...


In [15]:
d_2 = {'name':seq_ids,'sequence':seqs}

df2 = pd.DataFrame(d_2)
df2.head()

,name,sequence
0,sp|Q03518|TAP1_HUMAN,MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAE...
1,sp|Q03519|TAP2_HUMAN,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...
2,sp|Q9LVM1|AB25B_ARATH,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...
3,sp|Q9FUT3|AB23B_ARATH,MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFF...
4,sp|Q9NP58|ABCB6_HUMAN,MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVL...


In [16]:
df.shape

(4195, 2)

In [17]:
df2.shape

(4195, 2)

In [18]:
# remove non-coding amino acid letters like X,B,U
df['sequence'] = df['sequence'].str.replace('U','G')
df['sequence'] = df['sequence'].str.replace('X','G')
df['sequence'] = df['sequence'].str.replace('B','G')

In [19]:
# remove non-coding amino acid letters like X,B,U
df2['sequence'] = df2['sequence'].str.replace('U','G')
df2['sequence'] = df2['sequence'].str.replace('X','G')
df2['sequence'] = df2['sequence'].str.replace('B','G')

In [20]:
#df['MW'] = df['sequence'].apply(lambda seq: ProteinAnalysis(seq).molecular_weight())
#df.head()

In [21]:
#df2['MW'] = df2['sequence'].apply(lambda seq: ProteinAnalysis(seq).molecular_weight())
#df2.head()

In [22]:
# Create 2 columns for the dataframe
df['classification'] = 'TRANSPORT'
df['type'] = 10

df.head(10)

,name,sequence,classification,type
0,sp|Q03518|TAP1_HUMAN,MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAE...,TRANSPORT,10
1,sp|Q03519|TAP2_HUMAN,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,TRANSPORT,10
2,sp|Q9LVM1|AB25B_ARATH,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...,TRANSPORT,10
3,sp|Q9FUT3|AB23B_ARATH,MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFF...,TRANSPORT,10
4,sp|Q9NP58|ABCB6_HUMAN,MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVL...,TRANSPORT,10
5,sp|Q94FB9|AB1D_ARATH,MPSLQLLQLTERGRGLVASRRKSILLAAGIVAAGGTAVYLKSRVAS...,TRANSPORT,10
6,sp|Q5A762|MLT1_CANAL,MNESNRELILGLSPVHLSLFNSESLLHTFNFFGVGDGQANIASNYV...,TRANSPORT,10
7,sp|Q9UNQ0|ABCG2_HUMAN,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,TRANSPORT,10
8,sp|P33527|MRP1_HUMAN,MALRGFCSADGSDPLWDWNVTWNTSNPDFTKCFQNTVLVWVPCFYL...,TRANSPORT,10
9,sp|O35379|MRP1_MOUSE,MALRSFCSADGSDPLWDWNVTWHTSNPDFTKCFQNTVLTWVPCFYL...,TRANSPORT,10


In [23]:
df.shape

(4195, 4)

In [24]:
# remove the duplicate protein sequences
df = df.drop_duplicates(subset='sequence', keep="first")
df.shape

(3642, 4)

In [25]:
df.head(10)

,name,sequence,classification,type
0,sp|Q03518|TAP1_HUMAN,MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAE...,TRANSPORT,10
1,sp|Q03519|TAP2_HUMAN,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,TRANSPORT,10
2,sp|Q9LVM1|AB25B_ARATH,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...,TRANSPORT,10
3,sp|Q9FUT3|AB23B_ARATH,MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFF...,TRANSPORT,10
4,sp|Q9NP58|ABCB6_HUMAN,MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVL...,TRANSPORT,10
5,sp|Q94FB9|AB1D_ARATH,MPSLQLLQLTERGRGLVASRRKSILLAAGIVAAGGTAVYLKSRVAS...,TRANSPORT,10
6,sp|Q5A762|MLT1_CANAL,MNESNRELILGLSPVHLSLFNSESLLHTFNFFGVGDGQANIASNYV...,TRANSPORT,10
7,sp|Q9UNQ0|ABCG2_HUMAN,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,TRANSPORT,10
8,sp|P33527|MRP1_HUMAN,MALRGFCSADGSDPLWDWNVTWNTSNPDFTKCFQNTVLVWVPCFYL...,TRANSPORT,10
9,sp|O35379|MRP1_MOUSE,MALRSFCSADGSDPLWDWNVTWHTSNPDFTKCFQNTVLTWVPCFYL...,TRANSPORT,10


## Save dataframe of sequences as a csv file

In [26]:
df.to_csv('uniprot-ABC-Feb8.csv',index = None)

proteins = pd.read_csv('uniprot-ABC-Feb8.csv')

proteins.head()

,name,sequence,classification,type
0,sp|Q03518|TAP1_HUMAN,MAELLASAGSACSWDFPRAPPSFPPPAASRGGLGGTRSFRPHRGAE...,TRANSPORT,10
1,sp|Q03519|TAP2_HUMAN,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,TRANSPORT,10
2,sp|Q9LVM1|AB25B_ARATH,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...,TRANSPORT,10
3,sp|Q9FUT3|AB23B_ARATH,MMRGSRFLLSRASLYHRLRSDHHHHSFSSFIKRSSIQRSPAINAFF...,TRANSPORT,10
4,sp|Q9NP58|ABCB6_HUMAN,MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVL...,TRANSPORT,10


In [27]:
proteins.shape

(3642, 4)